In [1]:
# Importing the libraries
from PIL import Image
import numpy as np
import os 
import pandas as pd
import matplotlib.pyplot as plt
import cv2
#import serial 
#Arduino_Serial=serial.Serial('com4',9600)


In [ ]:
from tensorflow.keras.models import load_model
new_model=load_model("my_model.h5")

In [ ]:
# Doing some Face Recognition with the webcam
video_capture = cv2.VideoCapture(0)
while True:  
    # Read the frame  
    _, imgR = video_capture.read()  
    # Convert to grayscale  
    image1 = cv2.cvtColor(imgR, cv2.COLOR_BGR2RGB)
    image = cv2.cvtColor(image1, cv2.COLOR_RGB2HSV)
    (h,w)=image.shape[:2]
    cx,cy=(w//2),(h//2)
    left=image1[0:h,0:cx]
    right=image1[0:h,cx:w]
    imgR=cv2.rectangle(imgR,(0,0),(cx,h),(0,255,0),3)
    imgR=cv2.rectangle(imgR,(cx+5,0),(w,h),(255,0,0),3)
    lower = [np.mean(image[:,:,i] - np.std(image[:,:,i])/3 ) for i in range(3)]
    upper = [250, 250, 250]
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")
    mask = cv2.inRange(image, lower, upper)
    output = cv2.bitwise_and(image, image, mask=mask)
    ret,thresh = cv2.threshold(mask, 40, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    # find the biggest countour (c) by the area
    c = max(contours, key = cv2.contourArea)
    x,y,w,h = cv2.boundingRect(c)
    # draw the biggest contour (c) in green
    cv2.rectangle(output,(x,y),(x+w,y+h),(0,255,0),5)
    
    #for left nozzle
    left=cv2.resize(left,(256, 256))
    imgL= np.array(left)/255
    XL = image.img_to_array(imgL)
    valL= new_model.predict(XL)
    
    right=cv2.resize(right,(256, 256))
    imgR= np.array(right)/255
    XR = image.img_to_array(imgR)
    valR= new_model.predict(XR)
    
    if valL==0:
        cv2.putText(output, "Tobacco", (x+20, y-20),cv2.FONT_HERSHEY_SIMPLEX,1, (0, 0, 255), 3)
        Arduino_Serial.write('1') # Activate left nozzle solenoid valve 
        elif valL==1:
            cv2.putText(output, "Weed", (x+20, y-20),cv2.FONT_HERSHEY_SIMPLEX,1, (0, 0, 255), 3)
            
        
    
    if valR==0:
        cv2.putText(output, "Tobacco", (x+20, y-20),cv2.FONT_HERSHEY_SIMPLEX,1, (0, 0, 255), 3)
        Arduino_Serial.write('2') # Activate right nozzle solenoid valve 
        elif valR==1:
            cv2.putText(output, "Weed", (x+20, y-20),cv2.FONT_HERSHEY_SIMPLEX,1, (0, 0, 255), 3)
    
    

    cv2.imshow('output', output)  
    cv2.imshow('Video', imgR) 
    #cv2.imshow('left', left) 
    #cv2.imshow('right', right) 

   # Stop if escape key is pressed  
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release the VideoCapture object 

video_capture.release()
cv2.destroyAllWindows()